# Capture the Flag (RL - Policy Gradient)

- Seung Hyun Kim
- skim449@illinois.edu

## Implementation Details

- Actor-critic experience buffer.
- Off-policy: Training is done at the end of the rollout

### Sampling
- [ ] Mini-batch to update 'average' gradient
- [ ] Experience Replay for Random Sampling
- [ ] Importance Sampling
    
### Deterministic Policy Gradient
- [ ] DDPG
- [ ] MADDPG

### Stability and Reducing Variance
- [x] Gradient clipping
- [ ] Normalized Reward/Advantage
- [ ] Target Network
- [ ] TRPO
- [ ] PPO

### Multiprocessing
- [ ] Synchronous Training (A2C)
- [x] Asynchronous Training (A3C)

### Applied Training Methods:
- [ ] Self-play
- [ ] Batch Policy

## Notes

- This notebook includes:
    - Building the structure of policy driven network.
    - Training with/without render
    - Saver that save model and weights to ./model directory
    - Writer that will record some necessary datas to ./logs

- This notebook does not include:
    - Simulation with RL policy
        - The simulation can be done using policy_RL.py
    - cap_test.py is changed appropriately.
    
## References :
- https://github.com/awjuliani/DeepRL-Agents/blob/master/Vanilla-Policy.ipynb (source)
- https://www.youtube.com/watch?v=PDbXPBwOavc
- https://github.com/lilianweng/deep-reinforcement-learning-gym/blob/master/playground/policies/actor_critic.py (source)
- https://github.com/spro/practical-pytorch/blob/master/reinforce-gridworld/reinforce-gridworld.ipynb

## TODO:

- Research on '_bootstrap_' instead of end-reward

In [1]:
!rm -rf logs/B4R4_Rzero_A3C/ model/B4R4_Rzero_A3C

In [2]:
TRAIN_NAME='B4R4_Rzero_A3C'
LOG_PATH='./logs/'+TRAIN_NAME
MODEL_PATH='./model/' + TRAIN_NAME
GPU_CAPACITY=0.25 # gpu capacity in percentage

In [3]:
import os

import signal
import multiprocessing
import threading

import tensorflow as tf
import tensorflow.contrib.slim as slim
import tensorflow.contrib.layers as layers
from tensorflow.python.client import device_lib
import matplotlib.pyplot as plt
%matplotlib inline

import time
import gym
import gym_cap
import gym_cap.envs.const as CONST
import numpy as np
import random
import math

# the modules that you can use to generate the policy. 
import policy.random
import policy.roomba
import policy.policy_RL
import policy.zeros

# Data Processing Module
from utility.dataModule import one_hot_encoder
from utility.utils import MovingAverage as MA
from utility.utils import Experience_buffer, discount_rewards
from utility.vec_env import SubprocVecEnv

from network.ActorCritic import ActorCritic as AC

%load_ext autoreload
%autoreload 2

## Hyperparameters

In [4]:
# Replay Variables
total_episodes= 500000
max_ep = 150
update_frequency = 32
entropy_beta = 0.001

# Saving Related
save_network_frequency = 1200
save_stat_frequency = 128
moving_average_step = 128

# Training Variables
decay_lr = False
lr_a = 0.0001
lr_c = 0.001
lr_a_gamma = 0.9995
lr_c_gamma = 0.9995
lr_a_final = 1e-5
lr_c_final = 1e-4
lr_a_decay_step = (int) (math.log(lr_a_final/lr_a) / math.log(lr_a_gamma))
lr_c_decay_step = (int) (math.log(lr_c_final/lr_c) / math.log(lr_c_gamma))

gamma = 0.99 # discount_factor

# Env Settings
MAP_SIZE = 20
VISION_RANGE = 9 # What decide the network size !!!
VISION_dX, VISION_dY = 2*VISION_RANGE+1, 2*VISION_RANGE+1
# NENV = 4
in_size = [None,VISION_dX,VISION_dY,6]
NENV = multiprocessing.cpu_count()

# Asynch Settings
global_scope = 'global'

## Environment Setting

In [5]:
if not os.path.exists(MODEL_PATH):
    os.makedirs(MODEL_PATH)
    
#Create a directory to save episode playback gifs to
if not os.path.exists(LOG_PATH):
    os.makedirs(LOG_PATH)

In [6]:
action_space = 5
n_agent = 4

## A3C Network Structure

![Network Structure](https://cdn-images-1.medium.com/max/1600/1*YtnGhtSAMnnHSL8PvS7t_w.png)

- Network is given in network.ActorCritic

## Environments

![Environment Interaction Diagram](https://cdn-images-1.medium.com/max/1600/1*Hzql_1t0-wwDxiz0C97AcQ.png)

In [7]:
global_rewards = MA(moving_average_step)
global_ep_rewards = MA(moving_average_step)
global_length = MA(moving_average_step)
global_succeed = MA(moving_average_step)
global_episodes = 0

# Launch the session
gpu_options = tf.GPUOptions(per_process_gpu_memory_fraction=GPU_CAPACITY, allow_growth=True)

sess = tf.Session(config=tf.ConfigProto(gpu_options=gpu_options))
#sess = tf.Session()
progbar = tf.keras.utils.Progbar(total_episodes,interval=0.5)

In [8]:
class Worker(object):
    def __init__(self, name, globalAC, sess, global_step=0):
        # Initialize Environment worker
        self.env = gym.make("cap-v0").unwrapped
        self.name = name
        
        
        # Create AC Network for Worker
        self.AC = AC(in_size=in_size,
                     action_size=action_space,
                     decay_lr=decay_lr,
                     lr_actor=lr_a,
                     lr_critic=lr_c,
                     grad_clip_norm=0,
                     scope=self.name,
                     global_step=global_step,
                     initial_step=0,
                     entropy_beta = 0.001,
                     sess=sess,
                     globalAC=globalAC)
        self.sess=sess
        
    def work(self, saver, writer):
        global global_rewards, global_ep_rewards, global_episodes, global_length, global_succeed
        total_step = 1
                
        # loop
        with self.sess.as_default(), self.sess.graph.as_default():          
            while not coord.should_stop() and global_episodes < total_episodes:
                s0 = self.env.reset(map_size=MAP_SIZE, policy_red=policy.zeros.PolicyGen(self.env.get_map, self.env.get_team_red))
                #s0 = one_hot_encoder(s0, self.env.get_team_blue, VISION_RANGE)
                s0 = one_hot_encoder(self.env._env, self.env.get_team_blue, VISION_RANGE)
                # parameters 
                ep_r = 0 # Episodic Reward
                prev_r = 0
                was_alive = [ag.isAlive for ag in self.env.get_team_blue]
                
                indv_history = [ [] for _ in range(len(self.env.get_team_blue)) ]
                for step in range(max_ep+1):
                    a = self.AC.choose_action(s0).tolist()
                    print(a)

                    s1, rc, d, _ = self.env.step(a)
                    #s1 = one_hot_encoder(s1, self.env.get_team_blue, VISION_RANGE)
                    s1 = one_hot_encoder(self.env._env, self.env.get_team_blue, VISION_RANGE)
                    
                    r = rc - prev_r -1
                    d = (True if d or step == max_ep else False)
                    if step == max_ep and d == False:
                        '''r = -100
                        rc = -100'''
                        d = True
                        
                    ep_r += r
                    
                    if d:
                        v1 = [0.0 for _ in range(len(self.env.get_team_blue))]
                    else:
                        v1 = self.sess.run(self.AC.critic, {self.AC.state_input:s1})

                    # push to buffer (normalize reward)
                    for idx, agent in enumerate(self.env.get_team_blue):
                        if was_alive[idx]:
                            indv_history[idx].append([s0[idx],a[idx],r,v1[idx]])

                    if total_step % update_frequency == 0 or d:
                        feed_dict = self.train(indv_history, sess, v1)
                        indv_history = [ [] for _ in range(len(self.env.get_team_blue)) ]
                
                    # Iteration
                    prev_r = rc
                    was_alive = [ag.isAlive for ag in self.env.get_team_blue]
                    s0=s1
                    total_step += 1

                    if d:
                        global_ep_rewards.append(ep_r)
                        global_rewards.append(rc)
                        global_length.append(step)
                        global_succeed.append(self.env.blue_win)
                        global_episodes += 1
                        progbar.update(global_episodes)
                        if global_episodes % save_stat_frequency == 0 and global_episodes != 0:
                            summary = tf.Summary()
                            summary.value.add(tag='Records/mean_reward', simple_value=global_rewards())
                            summary.value.add(tag='Records/mean_length', simple_value=global_length())
                            summary.value.add(tag='Records/mean_succeed', simple_value=global_succeed())
                            writer.add_summary(summary,global_episodes)

                            #summary_str = sess.run(self.AC.summary_,feed_dict=feed_dict)
                            #writer.add_summary(summary_str, global_episodes)

                            writer.flush()
                        if global_episodes % save_network_frequency == 0 and global_episodes != 0:
                            saver.save(self.sess, MODEL_PATH+'/ctf_policy.ckpt', global_step=global_episodes)
                        break
                        
    def train(self, indv_buffer, sess, bootstrap=0.0):
        buffer_s, buffer_a, buffer_tdtarget, buffer_adv = [], [], [], []
        for idx, buffer in enumerate(indv_buffer):
            if len(buffer) == 0:
                continue
            _history = np.array(buffer)
            observations = _history[:,0]
            actions = _history[:,1]
            rewards = _history[:,2]
            values = _history[:,3]
            
            rewards_ext = np.append(rewards, [bootstrap[idx]])
            discounted_rewards = discount_rewards(rewards_ext,gamma)[:-1]
            value_ext = np.append(values, [bootstrap[idx]])
            td_target = rewards + gamma * value_ext[1:]
            advantages = rewards + gamma * value_ext[1:] - value_ext[:-1]
            #advantages = discount_rewards(advantages,gamma)
            
            buffer_s.extend(_history[:,0].tolist())
            buffer_a.extend(_history[:,1].tolist())
            buffer_tdtarget.extend(td_target.tolist())
            buffer_adv.extend(advantages.tolist())
            
        buffer_s, buffer_a, buffer_tdtarget, buffer_adv = np.stack(buffer_s), np.stack(buffer_a), np.stack(buffer_tdtarget), np.stack(buffer_adv)
        feed_dict = {
            self.AC.state_input      : buffer_s,
            self.AC.action_holder    : buffer_a,
            self.AC.td_target_holder : buffer_tdtarget,
            self.AC.advantage_holder : buffer_adv
        }

        # Update Buffer
        self.AC.update_global(feed_dict)

        # get global parameters to local ActorCritic 
        self.AC.pull_global()
        
        return feed_dict

In [9]:
with tf.device("/cpu:0"):
    # Global Network
    global_ac = AC(in_size=in_size, action_size=action_space, scope=global_scope,sess=sess)
    
    # Local workers
    workers = []
    # loop for each workers
    for idx in range(NENV):
        name = 'W_%i' % idx
        workers.append(Worker(name, global_ac, sess))
        print(f'worker: {name} initiated')
    saver = tf.train.Saver(max_to_keep=3)
    writer = tf.summary.FileWriter(LOG_PATH, sess.graph)
coord = tf.train.Coordinator()

sess.run(tf.global_variables_initializer())

ckpt = tf.train.get_checkpoint_state(MODEL_PATH)
if ckpt and tf.train.checkpoint_exists(ckpt.model_checkpoint_path):
    saver.restore(sess, ckpt.model_checkpoint_path)
    print("Load Model : ", ckpt.model_checkpoint_path)
else:
    sess.run(tf.global_variables_initializer())
    print("Initialized Variables")
    
worker_threads = []

for worker in workers:
    job = lambda: worker.work(saver, writer)
    t = threading.Thread(target=job)
    t.start()
    worker_threads.append(t)
coord.join(worker_threads)

worker: W_0 initiated
worker: W_1 initiated
worker: W_2 initiated
worker: W_3 initiated
worker: W_4 initiated
worker: W_5 initiated
worker: W_6 initiated
worker: W_7 initiated
worker: W_8 initiated
worker: W_9 initiated
worker: W_10 initiated
worker: W_11 initiated
worker: W_12 initiated
worker: W_13 initiated
worker: W_14 initiated
worker: W_15 initiated
worker: W_16 initiated
worker: W_17 initiated
worker: W_18 initiated
worker: W_19 initiated
worker: W_20 initiated
worker: W_21 initiated
worker: W_22 initiated
worker: W_23 initiated
Initialized Variables
[3, 3, 4, 1]
[2, 3, 3, 4]
[1, 2, 1, 1]
[3, 2, 1, 2]
[0, 1, 2, 0]
[2, 1, 3, 0]
[3, 2, 0, 3]
[0, 0, 2, 3]
[4, 4, 1, 0]
[1, 1, 3, 0]
[4, 3, 1, 4]
[0, 3, 3, 0]
[1, 1, 3, 3]
[4, 3, 0, 1]
[1, 2, 3, 3]
[0, 3, 0, 3]
[3, 2, 2, 4]
[2, 0, 3, 3]
[4, 2, 1, 3]
[3, 4, 4, 4]
[1, 0, 4, 0]
[1, 2, 0, 1]
[1, 1, 0, 2]
[0, 0, 2, 4]
[3, 2, 0, 4]
[0, 2, 4, 2]
[0, 2, 2, 2][0, 3, 0, 4]

[4, 4, 3, 1]
[3, 2, 4, 1]
[2, 2, 1, 2]
[1, 0, 2, 0][1, 1, 3, 1]

[1, 1, 

[3, 0, 3, 4][4, 3, 1, 0]
[4, 2, 4, 0]
[4, 0, 1, 0]


[4, 2, 1, 4]
[4, 3, 1, 4][4, 0, 3, 3][0, 3, 2, 3]
[0, 4, 0, 3][1, 4, 0, 0]
[4, 0, 3, 1]


[0, 1, 3, 3][1, 0, 3, 4]
[3, 3, 1, 3]
[1, 1, 1, 0]


[2, 2, 1, 0][1, 3, 1, 3][3, 0, 2, 2]
[0, 3, 4, 0]
[3, 2, 1, 0]


[4, 1, 3, 4][1, 1, 2, 3]

[2, 2, 1, 4]
[3, 0, 4, 3][2, 1, 0, 3]

[1, 1, 0, 4]
[0, 4, 3, 4]
[1, 2, 3, 2][3, 1, 4, 4][0, 1, 1, 3][1, 3, 4, 1][0, 0, 3, 2][0, 1, 3, 2]
[1, 3, 1, 0]


[0, 4, 1, 3]
[1, 1, 4, 0]
[2, 3, 0, 2]



[2, 1, 0, 3]
[3, 4, 2, 1]
[3, 0, 2, 4]
[3, 4, 3, 0][3, 4, 4, 4][3, 0, 3, 2]

[3, 0, 3, 0]
[0, 2, 0, 2]
[1, 2, 0, 2][3, 1, 0, 2][0, 1, 1, 0]

[3, 4, 0, 2]
[1, 0, 0, 0]
[0, 2, 2, 1]
[2, 1, 0, 4]
[1, 1, 2, 2]
[1, 4, 4, 1]

[2, 4, 0, 2]
[2, 2, 4, 0][1, 1, 0, 1][2, 4, 0, 1]
[4, 1, 1, 0][2, 1, 3, 4]
[4, 0, 4, 0]
[4, 2, 2, 2]


[1, 2, 4, 3]
[3, 2, 2, 2]
[4, 0, 3, 4]

[3, 0, 3, 0]
[4, 1, 3, 4]
[2, 4, 4, 4]
[0, 4, 4, 3]
[2, 0, 3, 1]
[0, 0, 4, 4][2, 3, 1, 1][0, 3, 2, 1]
[2, 0, 2, 4]
[0, 1, 4, 1]
[4, 1, 3, 3]
[4, 2, 3, 0][4

[4, 3, 3, 0]
[3, 2, 3, 4]
[1, 1, 4, 1][1, 2, 0, 0][0, 4, 1, 3]
[3, 1, 4, 3]
[3, 3, 2, 3]

[4, 0, 1, 3]

[0, 4, 4, 1][0, 4, 3, 2][4, 3, 2, 1]
[2, 3, 1, 1][0, 0, 2, 2]

[1, 3, 4, 3]
[0, 4, 2, 3]
[3, 0, 3, 0]
[4, 3, 0, 2][2, 3, 2, 0]
[4, 3, 1, 3]


[4, 2, 1, 0]
[0, 0, 3, 1]
[4, 2, 2, 0]
[1, 3, 2, 3]

[2, 0, 4, 1]
[3, 4, 0, 3]

[4, 0, 2, 1][2, 3, 2, 2][2, 4, 2, 2][2, 3, 2, 0][2, 4, 0, 0][3, 2, 2, 0]
[4, 0, 0, 1]
[1, 4, 2, 0]


[3, 2, 3, 2][3, 3, 3, 0]
[0, 4, 4, 1]
[4, 4, 1, 2]


     5/500000 [..............................] - ETA: 2247:55:58
[4, 1, 1, 2]

[1, 3, 3, 0][0, 1, 0, 4]

[1, 2, 3, 2]


[2, 0, 0, 2]
[4, 4, 1, 4][4, 0, 0, 0]
[4, 1, 4, 0]
[2, 2, 0, 4][0, 0, 4, 1]

[1, 1, 0, 2]
[2, 0, 1, 0]
[4, 0, 0, 2]
[0, 3, 2, 0]
[0, 4, 4, 1]
[2, 3, 0, 2]
[1, 2, 0, 1]
[4, 0, 0, 4]
[1, 1, 2, 0]
[1, 2, 4, 3]
[3, 1, 0, 1]
[0, 2, 2, 0]

[0, 0, 0, 2][3, 2, 2, 3]
[1, 0, 1, 3]
[2, 2, 1, 3]
[2, 2, 3, 3]
[0, 4, 2, 3]
[3, 1, 1, 3]
[4, 1, 4, 1]
[4, 4, 0, 4]
[1, 3, 2, 2]

[3, 0, 1, 1]
[4, 2, 3, 3]
[3, 2, 3, 


[1, 1, 1, 2]

[0, 2, 3, 2]
[1, 4, 0, 3]
[1, 1, 4, 1][4, 2, 1, 3]
[0, 2, 3, 2][3, 4, 1, 4]
[4, 4, 4, 4][3, 2, 3, 1]
[2, 3, 4, 0]

[1, 1, 0, 0]

[0, 2, 4, 1]
[0, 3, 1, 1][3, 0, 1, 1][4, 1, 2, 3][2, 2, 3, 2]

[3, 1, 2, 0]



[3, 1, 4, 2][2, 4, 0, 4]

[4, 3, 3, 0][3, 3, 1, 4][3, 3, 2, 0]
[4, 1, 1, 2][2, 1, 4, 3]

[1, 3, 3, 2][2, 4, 0, 4][1, 3, 0, 4]

[1, 0, 2, 1]
[2, 4, 0, 1]

[4, 0, 2, 3]
[0, 2, 0, 4]


[2, 1, 1, 0]
[1, 0, 3, 4][1, 0, 1, 1]
[4, 3, 2, 1]
[2, 2, 2, 1][4, 2, 3, 4]
[4, 2, 4, 0]


[2, 0, 2, 2][0, 2, 0, 2]
[4, 0, 3, 4]
[0, 2, 0, 1]
[3, 4, 1, 0]
[0, 0, 1, 0][4, 3, 2, 3]

[0, 0, 1, 0]

[2, 0, 4, 2]
[3, 0, 3, 2]
[1, 2, 0, 2]
[1, 4, 0, 0][0, 1, 3, 0]
[1, 0, 2, 3][3, 2, 3, 3]

[0, 0, 3, 3]

[4, 3, 3, 1]
[3, 0, 0, 3][4, 1, 0, 0][2, 3, 4, 3][0, 0, 1, 1]
[3, 0, 1, 1]
[3, 0, 0, 2]
[4, 4, 3, 4]
[4, 0, 2, 1]
[3, 4, 1, 3]

[0, 2, 2, 1][1, 3, 1, 4]

[2, 4, 3, 1]
[1, 4, 1, 4]
[2, 1, 4, 1][3, 4, 3, 3][0, 0, 4, 2]
[2, 1, 4, 0]


[0, 0, 2, 4][0, 0, 0, 2][3, 0, 2, 2]


[0, 2, 2, 0]
[3, 1, 3, 0]


[1, 2, 2, 1][1, 0, 0, 4]
[2, 2, 3, 3]

[2, 2, 3, 3][2, 4, 3, 0]

[2, 1, 3, 1][4, 4, 4, 0][3, 1, 2, 2][1, 4, 3, 2]
[4, 3, 0, 2]
[1, 4, 3, 4][2, 0, 1, 1]


[4, 1, 4, 4][3, 4, 2, 4]


[2, 0, 4, 3][1, 2, 1, 3][1, 1, 1, 0]

[3, 4, 4, 0][4, 3, 2, 1][1, 3, 2, 2]

[4, 2, 2, 0]

[4, 1, 0, 0][1, 4, 3, 4]

[1, 4, 0, 1]
[3, 3, 1, 2]
[4, 2, 4, 0][4, 0, 0, 0][4, 0, 2, 2]
[3, 2, 3, 1]



[4, 3, 2, 0]
[1, 0, 0, 0]
[0, 4, 3, 3]

[4, 3, 0, 0]
[4, 2, 1, 1][0, 3, 3, 2]

[1, 3, 0, 0][2, 4, 0, 0][4, 4, 0, 3]

[4, 0, 0, 4]
[4, 1, 2, 2][4, 0, 4, 1]


[1, 0, 1, 0]
[3, 4, 3, 2][2, 1, 1, 3][2, 1, 2, 2]

[4, 4, 3, 0][1, 3, 1, 0]
[4, 3, 4, 2]
[0, 1, 3, 1]
[4, 3, 2, 2]
[0, 4, 4, 0]


[2, 2, 1, 3][2, 1, 4, 1]

[2, 1, 2, 4]
[3, 0, 3, 0][4, 0, 1, 4][0, 0, 3, 1][4, 2, 0, 4][2, 2, 2, 3]


[1, 4, 1, 0][0, 2, 4, 3]

[4, 4, 3, 0][2, 4, 0, 4]
[4, 2, 2, 4][3, 4, 0, 4][2, 1, 4, 0]

[2, 3, 3, 1]
[0, 1, 1, 3]
[4, 0, 0, 3]
[1, 0, 1, 0]
[2, 3, 3, 2]
[0, 3, 4, 2]

[0, 0, 3, 2]

[1, 0, 0, 0]
[1, 4, 3, 2]


[2, 0, 3, 0][0, 4, 2, 1]

...

Exception in thread Thread-14:
Traceback (most recent call last):
  File "/usr/lib/python3.6/threading.py", line 916, in _bootstrap_inner
    self.run()
  File "/usr/lib/python3.6/threading.py", line 864, in run
    self._target(*self._args, **self._kwargs)
  File "<ipython-input-9-0e8cdc542171>", line 29, in <lambda>
    job = lambda: worker.work(saver, writer)
  File "<ipython-input-8-d0305de471b6>", line 42, in work
    s1, rc, d, _ = self.env.step(a)
  File "/home/namsong/github/ctf_public/venv/lib/python3.6/site-packages/gym_cap/envs/cap_env.py", line 349, in step
    self.team_blue[idx].move(self.ACTION[act], self._env, self.team_home)
IndexError: list index out of range



[0, 4, 1, 0]
[0, 3, 0, 4][0, 1, 0, 1]

[2, 3, 0, 0]
[0, 2, 0, 4]
[3, 2, 1, 3]
[0, 2, 3, 1]
[0, 0, 2, 4]
[3, 1, 3, 0]
[2, 0, 1, 3]
[1, 2, 1, 1]
[1, 1, 0, 4]
[4, 3, 0, 2]
[2, 3, 2, 0]
[5, 5, 5, 5]
[3, 0, 0, 0]
[0, 3, 2, 0]
[3, 2, 4, 0]
[3, 2, 1, 3][1, 3, 1, 2]

   130/500000 [..............................][0, 1, 3, 4]
[3, 3, 1, 2] - ETA: 194:50:38[1, 0, 0, 4]
[0, 1, 0, 0]

[5, 5, 5, 5]
[3, 0, 3, 2]
[2, 0, 1, 0]
[2, 3, 0, 2]


Exception in thread Thread-12:
Traceback (most recent call last):
  File "/usr/lib/python3.6/threading.py", line 916, in _bootstrap_inner
    self.run()
  File "/usr/lib/python3.6/threading.py", line 864, in run
    self._target(*self._args, **self._kwargs)
  File "<ipython-input-9-0e8cdc542171>", line 29, in <lambda>
    job = lambda: worker.work(saver, writer)
  File "<ipython-input-8-d0305de471b6>", line 42, in work
    s1, rc, d, _ = self.env.step(a)
  File "/home/namsong/github/ctf_public/venv/lib/python3.6/site-packages/gym_cap/envs/cap_env.py", line 349, in step
    self.team_blue[idx].move(self.ACTION[act], self._env, self.team_home)
IndexError: list index out of range

Exception in thread Thread-28:
Traceback (most recent call last):
  File "/usr/lib/python3.6/threading.py", line 916, in _bootstrap_inner
    self.run()
  File "/usr/lib/python3.6/threading.py", line 864, in run
    self._target(*self._args, **self._kwargs)
  File "<ipython-input-9-0e8cdc542171>", line 29, in <l

[0, 1, 4, 1]
[5, 5, 5, 5]
[5, 5, 5, 5]
[5, 5, 5, 5]
[5, 5, 5, 5]
[5, 5, 5, 5]
[5, 5, 5, 5]
[5, 5, 5, 5]
[5, 5, 5, 5]
[5, 5, 5, 5][5, 5, 5, 5]



Exception in thread Thread-20:
Traceback (most recent call last):
  File "/usr/lib/python3.6/threading.py", line 916, in _bootstrap_inner
    self.run()
  File "/usr/lib/python3.6/threading.py", line 864, in run
    self._target(*self._args, **self._kwargs)
  File "<ipython-input-9-0e8cdc542171>", line 29, in <lambda>
    job = lambda: worker.work(saver, writer)
  File "<ipython-input-8-d0305de471b6>", line 42, in work
    s1, rc, d, _ = self.env.step(a)
  File "/home/namsong/github/ctf_public/venv/lib/python3.6/site-packages/gym_cap/envs/cap_env.py", line 349, in step
    self.team_blue[idx].move(self.ACTION[act], self._env, self.team_home)
IndexError: list index out of range

Exception in thread Thread-15:
Traceback (most recent call last):
  File "/usr/lib/python3.6/threading.py", line 916, in _bootstrap_inner
    self.run()
  File "/usr/lib/python3.6/threading.py", line 864, in run
    self._target(*self._args, **self._kwargs)
  File "<ipython-input-9-0e8cdc542171>", line 29, in <l